Wild Blueberry Yield Prediction
Wild blueberry yield prediction using machine learning 

Kaggle Link: https://www.kaggle.com/datasets/saurabhshahane/wild-blueberry-yield-prediction

# 1. Imports

In [8]:
import pandas as pd
import plotly.express as px
import numpy as np
import cut_the_tails as ct
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from xgboost import XGBClassifier, XGBRegressor
#from sklearn.model_selection import LeaveOneOut,KFold
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split

In [9]:
df = pd.read_csv('../preprocessed_data_sets/Blueberry_Yield.csv')
target = 'yield'

In [10]:
features = ['clonesize',
 'honeybee',
 'bumbles',
 'andrena',
 'osmia',
 'MaxOfUpperTRange',
 'MinOfUpperTRange',
 'AverageOfUpperTRange',
 'MaxOfLowerTRange',
 'MinOfLowerTRange',
 'AverageOfLowerTRange',
 'RainingDays',
 'AverageRainingDays',
 'fruitset',
 'fruitmass',
 'seeds']

In [11]:
fig = px.histogram(df, x=target, nbins=40)
fig.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

# Cauda Ótima

In [ ]:
df.dropna(inplace=True)
df = df.reset_index(drop=True)

model = RandomForestRegressor(max_depth=5, random_state=0) 
classifier = RandomForestClassifier(max_depth=5, random_state=0)

x,fval = ct.get_cuts_direct_optimization(df, target, features, classifier, model, 'direct')

print("Final Result: ")
print(x, fval)

[0.5 0.5] 0.02492645361905111
[0.5        0.83333333] 0.02407882464644069
[0.16666667 0.5       ] 0.024945896774220416
[0.5        0.83333333] 0.02407882464644069
[0.16666667 0.5       ] 0.024945896774220416
[0.83333333 0.83333333] 0.025663518786333005
[0.16666667 0.83333333] 0.02468897253083445
[0.16666667 0.83333333] 0.02468897253083445
[0.16666667 0.16666667] 0.024823750020728074
[0.61111111 0.83333333] 0.024512144392547796
[0.38888889 0.83333333] 0.02479220295144866
[0.5        0.94444444] 0.02418801151612854
[0.5        0.72222222] 0.024355807751281298
[0.5        0.94444444] 0.02418801151612854
[0.5        0.72222222] 0.024355807751281298
[0.61111111 0.83333333] 0.024512144392547796
[0.38888889 0.83333333] 0.02479220295144866
[0.61111111 0.94444444] 0.02427447775489491
[0.38888889 0.94444444] 0.024797259587291617
[0.53703704 0.83333333] 0.0246408358662055
[0.46296296 0.83333333] 0.023544254712142904
[0.5        0.87037037] 0.023454925804268896
[0.5       0.7962963] 0.024405799916

In [ ]:
cdf = ct.split_by_quantile_class(df,target,[0.4,0.85])
cdf.drop('Row#', axis=1, inplace=True)

In [ ]:
X = cdf[features].to_numpy()
y_tail = cdf['tail_class'].to_numpy()
y = cdf[target].to_numpy()

###Using split###
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train_aux, X_test_aux, y_train_tail, y_test_tail = train_test_split(X, y_tail, test_size=0.2, random_state=0)



In [ ]:
#baseline = RandomForestRegressor(max_depth=5, random_state=0)
baseline = XGBRegressor(n_estimators=100, max_leaves=0, random_state = 0)
#baseline = DecisionTreeRegressor(random_state=0, max_depth=5) 

In [ ]:
#tail_classifier = ct.fit_tail_classifier(X,y_tail,RandomForestClassifier(max_depth=5, random_state=0))
#tail_classifier = ct.fit_tail_classifier(X,y_tail,XGBClassifier(n_estimators=100, max_leaves=0, random_state = 0))
#tail_classifier = ct.fit_tail_classifier(X,y_tail,DecisionTreeClassifier(random_state=0, max_depth=5)) 


###Using split###
#tail_classifier = ct.fit_tail_classifier(X_train,y_train_tail,RandomForestClassifier(max_depth=5, random_state=0))
tail_classifier = ct.fit_tail_classifier(X_train,y_train_tail,XGBClassifier(n_estimators=100, max_leaves=0, random_state = 0))
#tail_classifier = ct.fit_tail_classifier(X_train,y_train_tail,DecisionTreeClassifier(random_state=0, max_depth=5)) 

In [ ]:
#models = ct.fit_tail_models(X,y,y_tail,baseline)

###Using split###
models = ct.fit_tail_models(X_train,y_train,y_train_tail,baseline)

In [ ]:
#baseline.fit(X,y)
#y_base = baseline.predict(X)

###Using split###
baseline.fit(X_train,y_train)
y_base = baseline.predict(X_test)

In [ ]:
#y_tail = ct.batch_tail_predict(X,tail_classifier,models)

###Using split###
y_tail = ct.batch_tail_predict(X_test,tail_classifier,models)

In [ ]:
#print(mean_absolute_error(y_base,y))
#print(mean_absolute_error(y_tail,y))
#print(mean_absolute_error(y,y))

###Using split###
print(mean_absolute_error(y_base,y_test))
print(mean_absolute_error(y_tail,y_test))
print(mean_absolute_error(y_test,y_test))

124.54776199338939
115.15781155779244
0.0


In [ ]:
#print(mean_absolute_percentage_error(y_base,y))
#print(mean_absolute_percentage_error(y_tail,y))
#print(mean_absolute_percentage_error(y,y))

###Using split###
print(mean_absolute_percentage_error(y_base,y_test))
print(mean_absolute_percentage_error(y_tail,y_test))
print(mean_absolute_percentage_error(y_test,y_test))

0.02329216119407331
0.021082707782797624
0.0
